In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

21/08/25 09:38:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [4]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   39 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              2
Core(s) per socket:              4
Socket(s):                       1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           142
Model name:                      Intel(R) Core(TM) i7-8665U CPU @ 1.90GHz
Stepping:                        12
CPU MHz:                         2112.005
BogoMIPS:                        4224.01
Hypervisor vendor:               Microsoft
Virtualization type:             full
L1d cache:                       128 KiB
L1i cache:                       128 KiB
L2 cache:                        1 MiB
L3 cache:                        8 MiB
Vulnerability Itlb multihit:     KVM: Vulnerable
Vulnerability L1tf: 

In [537]:
train = sc.textFile('train.csv')

In [538]:
test = sc.textFile('test.csv')

In [683]:
print(f'{train.take(3), test.take(3)}')

([[842.0, 0.0, 2.2, 0.0, 1.0, 0.0, 7.0, 0.6, 188.0, 2.0, 2.0, 20.0, 756.0, 2549.0, 9.0, 7.0, 19.0, 0.0, 0.0, 1.0], [1021.0, 1.0, 0.5, 1.0, 0.0, 1.0, 53.0, 0.7, 136.0, 3.0, 6.0, 905.0, 1988.0, 2631.0, 17.0, 3.0, 7.0, 1.0, 1.0, 0.0], [563.0, 1.0, 0.5, 1.0, 2.0, 1.0, 41.0, 0.9, 145.0, 5.0, 6.0, 1263.0, 1716.0, 2603.0, 11.0, 2.0, 9.0, 1.0, 1.0, 0.0]], [[1043.0, 1.0, 1.8, 1.0, 14.0, 0.0, 5.0, 0.1, 193.0, 3.0, 16.0, 226.0, 1412.0, 3476.0, 12.0, 7.0, 2.0, 0.0, 1.0, 0.0], [841.0, 1.0, 0.5, 1.0, 4.0, 1.0, 61.0, 0.8, 191.0, 5.0, 12.0, 746.0, 857.0, 3895.0, 6.0, 0.0, 7.0, 1.0, 0.0, 0.0], [1807.0, 1.0, 2.8, 0.0, 1.0, 0.0, 27.0, 0.9, 186.0, 3.0, 4.0, 1270.0, 1366.0, 2396.0, 17.0, 10.0, 10.0, 0.0, 1.0, 1.0]])


In [539]:
train_header = train.first() 
train = train\
    .filter(lambda row: row != train_header)\
    .map(lambda row: [float(el) for el in row.split(',')[:-1]])

In [540]:
# Преобразуйте test
test_header = test.first() 
test = test\
    .filter(lambda row: row != test_header)\
    .map(lambda row: [float(el) for el in row.split(',')[1:]])

In [542]:
# Объедините train и test
data = train.union(test)
data_header = train_header.split(',')[:-1]

In [544]:
height_col = data_header.index('px_height')
width_col = data_header.index('px_width')
sc_h_col = data_header.index('sc_h')
sc_w_col = data_header.index('sc_w')

In [563]:
# Объедините train и test, в разрезе широкоформатные или нет (экран широкоформатный, если соотногшение >=16:9)
data.map(lambda row: (row[width_col], row[height_col])).filter(lambda row: row[1]!=0).map(lambda row: (('high_resolution' if row[0]/row[1] >= 1.77 else 'low_resolution' ), 1)).reduceByKey(lambda x, y: x + y).sortByKey().collect()

[('high_resolution', 1643), ('low_resolution', 1353)]

In [669]:
# Выведите отсортированное распределение плотности пикселей
import math
data.map(lambda row: (round((math.sqrt(row[width_col]**2 + row[height_col]**2) ) / (math.sqrt(row[sc_h_col]**2 + row[sc_w_col]**2))), 1)).reduceByKey(lambda x, y: x + y).sortByKey(True).collect()

[(22, 1),
 (24, 3),
 (25, 2),
 (26, 3),
 (27, 3),
 (28, 3),
 (29, 5),
 (30, 3),
 (31, 11),
 (32, 11),
 (33, 17),
 (34, 13),
 (35, 13),
 (36, 11),
 (37, 9),
 (38, 15),
 (39, 15),
 (40, 17),
 (41, 11),
 (42, 9),
 (43, 16),
 (44, 15),
 (45, 14),
 (46, 20),
 (47, 21),
 (48, 27),
 (49, 17),
 (50, 23),
 (51, 9),
 (52, 21),
 (53, 16),
 (54, 31),
 (55, 22),
 (56, 16),
 (57, 23),
 (58, 15),
 (59, 23),
 (60, 21),
 (61, 20),
 (62, 23),
 (63, 13),
 (64, 19),
 (65, 26),
 (66, 21),
 (67, 29),
 (68, 23),
 (69, 30),
 (70, 14),
 (71, 29),
 (72, 23),
 (73, 22),
 (74, 26),
 (75, 21),
 (76, 20),
 (77, 25),
 (78, 29),
 (79, 27),
 (80, 25),
 (81, 26),
 (82, 34),
 (83, 16),
 (84, 25),
 (85, 14),
 (86, 19),
 (87, 25),
 (88, 20),
 (89, 33),
 (90, 24),
 (91, 21),
 (92, 16),
 (93, 22),
 (94, 21),
 (95, 22),
 (96, 17),
 (97, 24),
 (98, 26),
 (99, 14),
 (100, 16),
 (101, 19),
 (102, 26),
 (103, 25),
 (104, 22),
 (105, 16),
 (106, 18),
 (107, 17),
 (108, 20),
 (109, 22),
 (110, 13),
 (111, 25),
 (112, 20),
 (113, 1

In [640]:
train_to_join = train.map(lambda row: (round((math.sqrt(row[width_col]**2 + row[height_col]**2) ) / (math.sqrt(row[sc_h_col]**2 + row[sc_w_col]**2))), 1))
test_to_join = test.map(lambda row: (round((math.sqrt(row[width_col]**2 + row[height_col]**2) ) / (math.sqrt(row[sc_h_col]**2 + row[sc_w_col]**2))), 1))

In [645]:
# объединение по разрешению
join_data_rate = train_to_join.cogroup(test_to_join)

In [678]:
join_data_rate.map(lambda row: (row[0], sum([len(el) for el in row[1]]) )).sortByKey(True).collect()

[(22, 1),
 (24, 3),
 (25, 2),
 (26, 3),
 (27, 3),
 (28, 3),
 (29, 5),
 (30, 3),
 (31, 11),
 (32, 11),
 (33, 17),
 (34, 13),
 (35, 13),
 (36, 11),
 (37, 9),
 (38, 15),
 (39, 15),
 (40, 17),
 (41, 11),
 (42, 9),
 (43, 16),
 (44, 15),
 (45, 14),
 (46, 20),
 (47, 21),
 (48, 27),
 (49, 17),
 (50, 23),
 (51, 9),
 (52, 21),
 (53, 16),
 (54, 31),
 (55, 22),
 (56, 16),
 (57, 23),
 (58, 15),
 (59, 23),
 (60, 21),
 (61, 20),
 (62, 23),
 (63, 13),
 (64, 19),
 (65, 26),
 (66, 21),
 (67, 29),
 (68, 23),
 (69, 30),
 (70, 14),
 (71, 29),
 (72, 23),
 (73, 22),
 (74, 26),
 (75, 21),
 (76, 20),
 (77, 25),
 (78, 29),
 (79, 27),
 (80, 25),
 (81, 26),
 (82, 34),
 (83, 16),
 (84, 25),
 (85, 14),
 (86, 19),
 (87, 25),
 (88, 20),
 (89, 33),
 (90, 24),
 (91, 21),
 (92, 16),
 (93, 22),
 (94, 21),
 (95, 22),
 (96, 17),
 (97, 24),
 (98, 26),
 (99, 14),
 (100, 16),
 (101, 19),
 (102, 26),
 (103, 25),
 (104, 22),
 (105, 16),
 (106, 18),
 (107, 17),
 (108, 20),
 (109, 22),
 (110, 13),
 (111, 25),
 (112, 20),
 (113, 1

In [682]:
# объединение по плотности
train_to_join_px = train.map(lambda row: (row[width_col], row[height_col])).filter(lambda row: row[1]!=0).map(lambda row: (('high_resolution' if row[0]/row[1] >= 1.77 else 'low_resolution' ), 1))
test_to_join_px = test.map(lambda row: (row[width_col], row[height_col])).filter(lambda row: row[1]!=0).map(lambda row: (('high_resolution' if row[0]/row[1] >= 1.77 else 'low_resolution' ), 1))
join_data_px = train_to_join_px.cogroup(test_to_join_px)
join_data_px.map(lambda row: (row[0], sum([len(el) for el in row[1]]) )).sortByKey(True).collect()

[('high_resolution', 1643), ('low_resolution', 1353)]

In [765]:
# Считываем данные
df_train = spark.read.csv('train.csv', header=True, inferSchema=True)
df_test = spark.read.csv('test.csv', header=True, inferSchema=True)

In [778]:
# Сохранял все колонки из обоих таблиц, так как в последнем задании нужен был ID
# Если требовались только общие колонки, можно было на предыдущем этапе дропнуть лишние колонки при загрузке
data = df_test.unionByName(df_train, allowMissingColumns=True)

In [851]:
from pyspark.sql.functions import col, round, pow, sqrt
# Создали колонки с соотношением сторон и ppi
data = data.withColumn('asc_rate', round((col('px_width') / col('px_height')), 2)).withColumn('ppi', round((sqrt(pow(data.px_width, 2) + pow(data.px_height,2)) ) / (sqrt(pow(data.sc_h, 2) + pow(data.sc_w, 2)))))

In [863]:
# Сделали новый датасет для сохранения. Здесь не очень понял про формат talk_time и его преобразования
# В этой колонке только Int и все меньше 24, предполагаю что это часы, соответственено к запрашиваему формату привести их не очень понятно как.
data_to_save = data.select('id', 'ppi', 'talk_time')

In [865]:
data_to_save.write.csv('train_test_result.csv', sep=',', header=True, encoding='cp1251')